In [ ]:
import json
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from fffunds.company import identify_company

FF_COMPS_PATH = Path("../data/ff_comps.csv")
COMPARISON_PATH = Path("../tests/ff_names_matched.csv")
FUND_HOLDINGS_PATH = Path("../data/mbie_fund_holdings.json")

In [ ]:
# Import with Pandas as real data may be more than just a list of names.
ff_comps = pd.read_csv(FF_COMPS_PATH, header=None, names=["name"])
ff_names = ff_comps["name"].to_list()
display(ff_names)

In [ ]:
with FUND_HOLDINGS_PATH.open() as fp:
    fund_data = json.load(fp)

# Convert nested data into a single table.
holdings = pd.DataFrame(
    [
        [fund["fund_name"], holding["holding"], holding["amount"]]
        for fund in fund_data
        for holding in fund["holdings"]
    ],
    columns=["fund", "company_name", "amount"],
)

holdings["identified"] = holdings["company_name"].apply(
    lambda x: identify_company(x, ff_names, strict=False)
)
holdings["is_ff"] = ~holdings["identified"].isna()

display(holdings)

In [ ]:
# calculate proportion going to fossil fuel companies
fund_res = holdings.groupby("fund").agg(total=("amount", "sum"))
fund_res["ff_amount"] = (
    holdings[holdings["is_ff"]].groupby("fund").agg({"amount": "sum"})
)
fund_res["prop"] = fund_res["ff_amount"] / fund_res["total"]

# descending makes bars in chart below ascending from top to bottom
fund_res = fund_res.sort_values("prop", ascending=False)
display(fund_res)

## Data visualisation

In [ ]:
fig, ax = plt.subplots()
ax.barh(np.arange(len(fund_res)), 100 * fund_res["prop"])
ax.set_yticks(np.arange(len(fund_res)), labels=fund_res.index)
ax.set_xlabel("Proportion of funds going to fossil fuel companies")
ax.xaxis.set_major_formatter(mtick.PercentFormatter())

# Make labels for total amount going to fossil fuels
labels = [f"${amount/1e6:.1f}m" for amount in fund_res["ff_amount"]]
ax.bar_label(
    ax.containers[0],
    labels=labels,
    label_type="center",
    color="white",
    weight="bold",
);